In [1]:
import pandas as pd
import os
os.chdir(r"C:\Users\7LStar\Desktop\study\python\用户分层实战")
df=pd.read_excel('主订单数据集.xlsx')

In [2]:
df.head()

,品牌名,店铺名称,主订单编号,用户ID,付款时间,订单状态,实付金额,购买数量
0,阿粥（小z）,数据不吹牛,73465136654,uid135460366,2023-01-01 09:32:12,交易成功,166,1
1,阿粥（小z）,数据不吹牛,73465136655,uid135460367,2023-01-01 09:11:50,交易成功,117,1
2,阿粥（小z）,数据不吹牛,73465136656,uid135460368,2023-01-01 11:49:02,交易成功,166,1
3,阿粥（小z）,数据不吹牛,73465136657,uid135460369,2023-01-01 12:20:24,交易成功,77,1
4,阿粥（小z）,数据不吹牛,73465136658,uid135460370,2023-01-01 01:23:15,交易成功,158,1


In [3]:
df=df.dropna()
#异常值处理
up=df.describe()['实付金额']['75%']+(df.describe()['实付金额']['75%']-df.describe()['实付金额']['25%'])*3
low=df.describe()['实付金额']['25%']-(df.describe()['实付金额']['75%']-df.describe()['实付金额']['25%'])*3

df=df[df["实付金额"]<=up]

df=df[df["购买数量"]<=5]
df.head()

,品牌名,店铺名称,主订单编号,用户ID,付款时间,订单状态,实付金额,购买数量
0,阿粥（小z）,数据不吹牛,73465136654,uid135460366,2023-01-01 09:32:12,交易成功,166,1
1,阿粥（小z）,数据不吹牛,73465136655,uid135460367,2023-01-01 09:11:50,交易成功,117,1
2,阿粥（小z）,数据不吹牛,73465136656,uid135460368,2023-01-01 11:49:02,交易成功,166,1
3,阿粥（小z）,数据不吹牛,73465136657,uid135460369,2023-01-01 12:20:24,交易成功,77,1
4,阿粥（小z）,数据不吹牛,73465136658,uid135460370,2023-01-01 01:23:15,交易成功,158,1


# 计算RFM值

In [6]:
df_rfm=df[['用户ID','付款时间','实付金额']]
df_rfm.head()

,用户ID,付款时间,实付金额
0,uid135460366,2023-01-01 09:32:12,166
1,uid135460367,2023-01-01 09:11:50,117
2,uid135460368,2023-01-01 11:49:02,166
3,uid135460369,2023-01-01 12:20:24,77
4,uid135460370,2023-01-01 01:23:15,158


In [7]:
r=df_rfm.groupby("用户ID")['付款时间'].max().reset_index()
r.head()

,用户ID,付款时间
0,uid135460366,2023-01-01 09:32:12
1,uid135460367,2023-01-01 09:11:50
2,uid135460368,2023-04-07 18:19:37
3,uid135460369,2023-01-13 13:48:02
4,uid135460370,2023-06-16 08:05:47


In [12]:
import datetime as dt
r['R']=(pd.to_datetime('2024-1-1') - pd.to_datetime(r['付款时间'])).dt.days
r=r[["用户ID","R"]]
r.head()

,用户ID,R
0,uid135460366,364
1,uid135460367,364
2,uid135460368,268
3,uid135460369,352
4,uid135460370,198


In [37]:
df_rfm['日期标签']=df_rfm['付款时间'].astype(str).str[:10]
dup_f=df_rfm.groupby(['用户ID','日期标签'])['付款时间'].count().reset_index()

f=dup_f.groupby('用户ID')['付款时间'].count().reset_index()
f.columns=["用户ID","F"]
f.head()



C:\Users\7LStar\AppData\Local\Temp\ipykernel_22708\3109918539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rfm['日期标签']=df_rfm['付款时间'].astype(str).str[:10]


,用户ID,F
0,uid135460366,1
1,uid135460367,1
2,uid135460368,2
3,uid135460369,2
4,uid135460370,7


In [35]:
sum_m=df_rfm.groupby('用户ID')['实付金额'].sum().reset_index()
sum_m.columns=['用户ID','总支付金额']
sum_m.head()

,用户ID,总支付金额
0,uid135460366,166
1,uid135460367,117
2,uid135460368,241
3,uid135460369,417
4,uid135460370,1846


In [38]:
#合并表
com_m=pd.merge(sum_m,f,left_on='用户ID',right_on='用户ID',how='inner')
com_m['M']=com_m['总支付金额']/com_m['F']
com_m.head()

,用户ID,总支付金额,F,M
0,uid135460366,166,1,166.000000
1,uid135460367,117,1,117.000000
2,uid135460368,241,2,120.500000
3,uid135460369,417,2,208.500000
4,uid135460370,1846,7,263.714286


In [42]:
rfm=pd.merge(com_m,r,left_on='用户ID',right_on='用户ID',how='inner')
rfm=rfm.drop("总支付金额",axis=1)
rfm_copy=rfm.copy()
rfm.head()

,用户ID,F,M,R
0,uid135460366,1,166.000000,364
1,uid135460367,1,117.000000,364
2,uid135460368,2,120.500000,268
3,uid135460369,2,208.500000,352
4,uid135460370,7,263.714286,198


# 维度打分

R值代表了用户最后一次下单距离今天的天数，这个值越大，用户流失的可能性就越大。我们当然不希望用
户流失，所以R值越大，赋予的得分越低。<br/>
F值代表了用户购买频次，M值则是用户平均支付金额，这两个指标越大越好，即数值越大，得分越高。<br/>
RFM模型中打分一般采取5分制，有两种比较常见的打分方式，一种是按照数据的分位数来打分，另一种是
依据数据和业务的理解进行分值的划分。

# 分值计算

In [45]:
rfm['R-SCORE']=pd.cut(rfm['R'],bins=[0,30,60,120,180,1000000],labels=[5,4,3,2,1],right=False).astype(float)
rfm['F-SCORE']=pd.cut(rfm['F'],bins=[1,2,3,4,5,1000000],labels=[1,2,3,4,5],right=False).astype(float)
rfm['M-SCORE']=pd.cut(rfm['M'],bins=[0,50,100,150,200,1000000],labels=[1,2,3,4,5],right=False).astype(float)

rfm.head()

,用户ID,F,M,R,R-SCORE,F-SCORE,M-SCORE
0,uid135460366,1,166.000000,364,1.0,1.0,4.0
1,uid135460367,1,117.000000,364,1.0,1.0,3.0
2,uid135460368,2,120.500000,268,1.0,2.0,3.0
3,uid135460369,2,208.500000,352,1.0,2.0,5.0
4,uid135460370,7,263.714286,198,1.0,5.0,5.0


In [46]:
rfm['R是否大于均值']=(rfm['R-SCORE']>rfm['R-SCORE'].mean())*1
rfm['F是否大于均值']=(rfm['F-SCORE']>rfm['F-SCORE'].mean())*1
rfm['M是否大于均值']=(rfm['M-SCORE']>rfm['M-SCORE'].mean())*1
rfm.head()

,用户ID,F,M,R,R-SCORE,F-SCORE,M-SCORE,R是否大于均值,F是否大于均值,M是否大于均值
0,uid135460366,1,166.000000,364,1.0,1.0,4.0,0,0,1
1,uid135460367,1,117.000000,364,1.0,1.0,3.0,0,0,0
2,uid135460368,2,120.500000,268,1.0,2.0,3.0,0,1,0
3,uid135460369,2,208.500000,352,1.0,2.0,5.0,0,1,1
4,uid135460370,7,263.714286,198,1.0,5.0,5.0,0,1,1


In [47]:
rfm['人气数值']=(rfm['R是否大于均值']*100)+(rfm['F是否大于均值']*10)+(rfm['M是否大于均值']*1)
rfm.head()

,用户ID,F,M,R,R-SCORE,F-SCORE,M-SCORE,R是否大于均值,F是否大于均值,M是否大于均值,人气数值
0,uid135460366,1,166.000000,364,1.0,1.0,4.0,0,0,1,1
1,uid135460367,1,117.000000,364,1.0,1.0,3.0,0,0,0,0
2,uid135460368,2,120.500000,268,1.0,2.0,3.0,0,1,0,10
3,uid135460369,2,208.500000,352,1.0,2.0,5.0,0,1,1,11
4,uid135460370,7,263.714286,198,1.0,5.0,5.0,0,1,1,11


In [48]:
def transform_label(x):
    if x==111:
        label='重要价值用户'
    elif x==110:
        label='消费潜力用户'
    elif x==101:
        label='频次深耕用户'
    elif x==100:
        label='新用户'
    elif x==11:
        label='重要价值流失预警用户'
    elif x==10:
        label='一般用户'
    elif x==1:
        label='高消费唤回用户'
    elif x==0:
        label='流失用户'
    return label

In [49]:
rfm['人群类型']=rfm['人气数值'].apply(transform_label)
rfm.head()

,用户ID,F,M,R,R-SCORE,F-SCORE,M-SCORE,R是否大于均值,F是否大于均值,M是否大于均值,人气数值,人群类型
0,uid135460366,1,166.000000,364,1.0,1.0,4.0,0,0,1,1,高消费唤回用户
1,uid135460367,1,117.000000,364,1.0,1.0,3.0,0,0,0,0,流失用户
2,uid135460368,2,120.500000,268,1.0,2.0,3.0,0,1,0,10,一般用户
3,uid135460369,2,208.500000,352,1.0,2.0,5.0,0,1,1,11,重要价值流失预警用户
4,uid135460370,7,263.714286,198,1.0,5.0,5.0,0,1,1,11,重要价值流失预警用户


# 数据分析

In [50]:
#人群占比
count=rfm["人群类型"].value_counts().reset_index()
count.columns=["人群类型",'人数']
count["人群占比"]=count["人数"]/count["人数"].sum()
count

,人群类型,人数,人群占比
0,频次深耕用户,53509,0.340876
1,高消费唤回用户,21387,0.136245
2,重要价值用户,21178,0.134913
3,流失用户,20933,0.133352
4,新用户,18965,0.120815
5,重要价值流失预警用户,8585,0.054690
6,一般用户,6219,0.039618
7,消费潜力用户,6199,0.039490


In [54]:
#消费金额占比
rfm["购买总金额"]=rfm["M"]*rfm["F"]
mon=rfm.groupby("人群类型")["购买总金额"].sum().sort_values(ascending=False).reset_index()
mon.columns=['人群类型','消费金额']
mon['消费金额占比']=mon['消费金额']/mon['消费金额'].sum()
mon

,人群类型,消费金额,消费金额占比
0,重要价值用户,19195113.0,0.338591
1,频次深耕用户,17210690.0,0.303587
2,高消费唤回用户,6295500.0,0.111049
3,重要价值流失预警用户,6252058.0,0.110283
4,消费潜力用户,2199253.0,0.038794
5,新用户,1945283.0,0.034314
6,流失用户,1892981.0,0.033391
7,一般用户,1700278.0,0.029992


# 利用机器学习完成RFM模型

In [58]:
#计算首次购买时间距今多少天
r_first=df_rfm.groupby("用户ID")['付款时间'].min().reset_index()
r_first['L']=(pd.to_datetime('2024-1-1')-pd.to_datetime(r_first['付款时间'])).dt.days
rfm_v2=pd.merge(r_first[['用户ID','L']],rfm_copy,left_on='用户ID',right_on='用户ID',how='inner')
rfm_v2.head()

,用户ID,L,F,M,R
0,uid135460366,364,1,166.000000,364
1,uid135460367,364,1,117.000000,364
2,uid135460368,364,2,120.500000,268
3,uid135460369,364,2,208.500000,352
4,uid135460370,364,7,263.714286,198


In [59]:
rfm_v2['R-ZSCORE']=(rfm_v2['R']-rfm_v2['R'].mean())/rfm_v2['R'].std()
rfm_v2['F-ZSCORE']=(rfm_v2['F']-rfm_v2['F'].mean())/rfm_v2['F'].std()
rfm_v2['F-ZSCORE']=(rfm_v2['M']-rfm_v2['M'].mean())/rfm_v2['M'].std()
rfm_v2['L-ZSCORE']=(rfm_v2['L']-rfm_v2['L'].mean())/rfm_v2['L'].std()
rfm_v2.head()

,用户ID,L,F,M,R,R-ZSCORE,F-ZSCORE,M-ZSCORE,L-ZSCORE
0,uid135460366,364,1,166.000000,364,2.880647,-0.402362,-0.412474,2.288834
1,uid135460367,364,1,117.000000,364,2.880647,-0.402362,-0.703528,2.288834
2,uid135460368,364,2,120.500000,268,1.758568,0.328716,-0.682739,2.288834
3,uid135460369,364,2,208.500000,352,2.740387,0.328716,-0.160029,2.288834
4,uid135460370,364,7,263.714286,198,0.940385,3.984106,0.167937,2.288834


In [61]:
#导入k-means模型
from sklearn.cluster import KMeans

kmeans=KMeans(n_clusters=6,random_state=0)

y_pred=kmeans.fit_predict(rfm_v2[['R-ZSCORE','F-ZSCORE','M-ZSCORE','L-ZSCORE']])

print(y_pred)

[2 2 2 ... 3 3 5]


In [63]:
kmeans_res=pd.DataFrame(kmeans.cluster_centers_,index=['用户类别1','用户类别2','用户类别3','用户类别4','用户类别5','用户类别6'],
                       columns=['R-ZSCORE','F-ZSCORE','M-ZSCORE','L-ZSCORE'])
kmeans_res

,R-ZSCORE,F-ZSCORE,M-ZSCORE,L-ZSCORE
用户类别1,-0.302903,1.316787,-0.127387,1.073386
用户类别2,0.169373,-0.307049,-0.399770,-0.079020
用户类别3,1.713954,-0.223212,-0.204272,1.368920
用户类别4,-0.899246,-0.227791,-0.111871,-0.980603
用户类别5,-0.556586,4.804622,-0.055788,1.529056
用户类别6,-0.327234,-0.275224,2.092177,-0.473589


用户类别3的R-SCORE很大，表示这群用户最近没有消费；M-SCORE偏小，说明他们的平均消费金
额不高；L-SCORE偏大，表明他们是很早进行首次消费的用户。这类用户可以概括为早期低客单价消费且
流失的用户。